In [4]:
import pandas as pd
import os
import sqlite3
import numpy as np
import joblib
import ast
from datetime import datetime
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Provided functions (unchanged)
def custom_tokenizer(text):
    if not isinstance(text, str) or not text.strip():
        return []
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english')) - {'not', 'no', 'never'}  # Exclude important words
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2 and token not in ['example', 'test']]  # Ignore short and specific words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]  # Lemmatization
    return tokens

def preprocess_text(text):
    return custom_tokenizer(text)

def load_embeddings(dataset_name):
    embedding_file = f"data/{dataset_name}/embeddings_matrix.joblib"
    if os.path.exists(embedding_file):
        return joblib.load(embedding_file)
    else:
        raise FileNotFoundError(f"Embedding file {embedding_file} not found. Please generate embeddings first.")

def enhance_query(query, dataset_name, model, enhance=True):
    processed_query = preprocess_text(query)
    
    if enhance:
        query_embeddings = load_embeddings(dataset_name)
        if len(query_embeddings) == 0:
            print(f"No query embeddings available for {dataset_name}")
            return query, processed_query

        query_text = ' '.join(processed_query)
        query_vector = model.encode([query_text], convert_to_numpy=True)
        similarities = cosine_similarity(query_vector, query_embeddings)[0]
        top_indices = np.argsort(similarities)[::-1][:5]

        db_file = f"data/{dataset_name}/index.db"
        if not os.path.exists(db_file):
            print(f"Error: Database file {db_file} not found")
            return query, processed_query

        mapping_file = f"data/{dataset_name}/doc_id_mapping.joblib"
        if not os.path.exists(mapping_file):
            print(f"Error: Mapping file {mapping_file} not found")
            return query, processed_query

        try:
            index_to_doc_id = joblib.load(mapping_file)
        except Exception as e:
            print(f"Error loading doc_id_mapping: {str(e)}")
            return query, processed_query

        try:
            conn = sqlite3.connect(db_file)
            cursor = conn.cursor()
            cursor.execute('SELECT doc_id, processed_text FROM documents')
            doc_data = {row[0]: row[1] for row in cursor.fetchall()}
            conn.close()

            if not doc_data:
                print(f"Warning: No processed documents found in database for {dataset_name}")
                return query, processed_query

            print(f"Total documents in doc_data: {len(doc_data)}")
            print(f"Sample doc_ids: {list(doc_data.keys())[:5]}")
            print(f"Top indices: {top_indices}")

            similar_queries = []
            max_add = 5
            added = 0
            top_doc_words = []

            for idx in top_indices:
                doc_id = index_to_doc_id.get(idx)
                if doc_id is None:
                    print(f"Warning: No doc_id mapped for index {idx}")
                    continue

                if doc_id in doc_data:
                    try:
                        sim_score = similarities[idx]
                        if 0.4 < sim_score < 0.999:
                            processed_query_text = ast.literal_eval(doc_data[doc_id]) if doc_data[doc_id] else []
                            if not processed_query_text:
                                print(f"Warning: Empty processed_text for doc_id {doc_id}")
                                continue
                            similar_queries.append(processed_query_text)
                            top_doc_words.extend(processed_query_text)
                            added += 1
                            if added >= max_add:
                                break
                    except (ValueError, SyntaxError) as e:
                        print(f"Error parsing processed_text at doc_id {doc_id}: {str(e)}")
                        continue
                else:
                    print(f"Warning: doc_id {doc_id} not found in doc_data")

            if not similar_queries:
                print(f"No similar documents found for {dataset_name} with sufficient similarity")
                return query, processed_query

            new_words = []
            seen = set(processed_query)
            for word in top_doc_words:
                if word not in seen and word not in new_words:
                    new_words.append(word)
                    if len(new_words) >= 4:
                        break

            additional_words = ' '.join(new_words)
            print(f"Additional words from top 5 documents: {additional_words}")

            if additional_words:
                enhanced_query = query + ' ' + additional_words
                print(f"🔍 Enhanced Query: {enhanced_query}")
            else:
                enhanced_query = query
                print(f"No new words to add to query for {dataset_name}")

        except Exception as e:
            print(f"Error enhancing query for {dataset_name}: {str(e)}")
            if 'conn' in locals():
                conn.close()
            enhanced_query = query

        return enhanced_query, processed_query

    return query, processed_query

def process_csv(input_csv, output_csv, dataset_name, model):
    # Ensure logs directory exists
    os.makedirs("logs", exist_ok=True)
    
    # Read the input CSV (limit to first 1000 queries)
    try:
        df = pd.read_csv(input_csv).head(1000)  # ✅ فقط أول 1000 استعلام
        print(f"Loaded first 1000 queries from: {input_csv}")
    except Exception as e:
        print(f"Error reading CSV file {input_csv}: {str(e)}")
        return

    # Ensure the text column exists
    if 'text' not in df.columns:
        print("Error: 'text' column not found in CSV")
        return

    # Process each query with tqdm progress bar
    enhanced_queries = []
    processed_queries = []
    
    for query in tqdm(df['text'], desc="معالجة الاستعلامات", unit="استعلام"):
        enhanced_query, processed_query = enhance_query(query, dataset_name, model, enhance=True)
        enhanced_queries.append(enhanced_query)
        processed_queries.append(' '.join(processed_query))

    # Create a new DataFrame with the same columns
    result_df = df.copy()
    result_df['text'] = enhanced_queries
    result_df['processed_query'] = processed_queries

    # Save to output CSV
    try:
        result_df.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"Enhanced queries saved to {output_csv}")
    except Exception as e:
        print(f"Error saving CSV file {output_csv}: {str(e)}")

# Example usage
if __name__ == "__main__":
    from sentence_transformers import SentenceTransformer
    
    # Load the model
    model = SentenceTransformer('all-mpnet-base-v2')
    dataset_name = "beir"
    input_csv = r"data\beir\queries_beir.csv"
    output_csv = "enhanced_queries_beir.csv"
    
    process_csv(input_csv, output_csv, dataset_name, model)

Loaded first 1000 queries from: data\beir\queries_beir.csv


معالجة الاستعلامات:   0%|          | 1/1000 [00:04<1:16:08,  4.57s/استعلام]

Total documents in doc_data: 522931
Sample doc_ids: ['1', '2', '3', '4', '5']
Top indices: [263218  61789 182493  51977  18681]
Additional words from top 5 documents: write
🔍 Enhanced Query: Which question should I ask on Quora? write


معالجة الاستعلامات:   0%|          | 2/1000 [00:08<1:13:24,  4.41s/استعلام]

Total documents in doc_data: 522931
Sample doc_ids: ['1', '2', '3', '4', '5']
Top indices: [ 66647 311735 250612 250613 422455]
Additional words from top 5 documents: nightmares dream avoid
🔍 Enhanced Query: What causes a nightmare? nightmares dream avoid


...

معالجة الاستعلامات: 100%|█████████▉| 999/1000 [1:01:14<00:03,  3.52s/استعلام]

Total documents in doc_data: 522931
Sample doc_ids: ['1', '2', '3', '4', '5']
Top indices: [501127 500200  47594  99929 500201]
Additional words from top 5 documents: traveller travel tourism resident
🔍 Enhanced Query: What is the difference between Traveler and Tourist? traveller travel tourism resident


معالجة الاستعلامات: 100%|██████████| 1000/1000 [1:01:18<00:00,  3.68s/استعلام]

Total documents in doc_data: 522931
Sample doc_ids: ['1', '2', '3', '4', '5']
Top indices: [484783 129011 235755 201750 275273]
Additional words from top 5 documents: practically limit faster impossible
🔍 Enhanced Query: Can you exceed the speed of light? practically limit faster impossible


Enhanced queries saved to enhanced_queries_beir.csv


In [3]:
import pandas as pd
import os
import sqlite3
import numpy as np
import joblib
import ast
from datetime import datetime
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Provided functions (unchanged)
def custom_tokenizer(text):
    if not isinstance(text, str) or not text.strip():
        return []
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english')) - {'not', 'no', 'never'}  # Exclude important words
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2 and token not in ['example', 'test']]  # Ignore short and specific words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]  # Lemmatization
    return tokens

def preprocess_text(text):
    return custom_tokenizer(text)

def load_embeddings(dataset_name):
    embedding_file = f"data/{dataset_name}/embeddings_matrix.joblib"
    if os.path.exists(embedding_file):
        return joblib.load(embedding_file)
    else:
        raise FileNotFoundError(f"Embedding file {embedding_file} not found. Please generate embeddings first.")

def enhance_query(query, dataset_name, model, enhance=True):
    processed_query = preprocess_text(query)
    log_file = f"logs/preproc_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    
    if enhance:
        query_embeddings = load_embeddings(dataset_name)
        if len(query_embeddings) == 0:
            print(f"No query embeddings available for {dataset_name}")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"No query embeddings available for {dataset_name} at {datetime.now()}\n")
            return query, processed_query

        query_text = ' '.join(processed_query)
        query_vector = model.encode([query_text], convert_to_numpy=True)
        similarities = cosine_similarity(query_vector, query_embeddings)[0]
        top_indices = np.argsort(similarities)[::-1][:5]
        print(f"Top similarities: {similarities[top_indices]}")
        with open(log_file, "a", encoding="utf-8") as log:
            log.write(f"Top similarities for {dataset_name}: {similarities[top_indices]} at {datetime.now()}\n")

        db_file = f"data/{dataset_name}/index.db"
        if not os.path.exists(db_file):
            print(f"Error: Database file {db_file} not found")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"Error: Database file {db_file} not found at {datetime.now()}\n")
            return query, processed_query

        mapping_file = f"data/{dataset_name}/doc_id_mapping.joblib"
        if not os.path.exists(mapping_file):
            print(f"Error: Mapping file {mapping_file} not found")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"Error: Mapping file {mapping_file} not found at {datetime.now()}\n")
            return query, processed_query

        try:
            index_to_doc_id = joblib.load(mapping_file)
            print(f"Loaded doc_id_mapping with {len(index_to_doc_id)} entries")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"Loaded doc_id_mapping with {len(index_to_doc_id)} entries at {datetime.now()}\n")
        except Exception as e:
            print(f"Error-Error loading doc_id: {str(e)}")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"Error loading doc_id_mapping: {str(e)} at {datetime.now()}\n")
            return query, processed_query

        try:
            conn = sqlite3.connect(db_file)
            cursor = conn.cursor()
            cursor.execute('SELECT doc_id, processed_text FROM documents')
            doc_data = {row[0]: row[1] for row in cursor.fetchall()}
            conn.close()

            if not doc_data:
                print(f"Warning: No processed documents found in database for {dataset_name}")
                with open(log_file, "a", encoding="utf-8") as log:
                    log.write(f"No processed documents found in database for {dataset_name} at {datetime.now()}\n")
                return query, processed_query

            print(f"Total documents in doc_data: {len(doc_data)}")
            print(f"Sample doc_ids: {list(doc_data.keys())[:5]}")
            print(f"Top indices: {top_indices}")

            similar_queries = []
            max_add = 5
            added = 0
            top_doc_words = []

            for idx in top_indices:
                doc_id = index_to_doc_id.get(idx)
                if doc_id is None:
                    print(f"Warning: No doc_id mapped for index {idx}")
                    with open(log_file, "a", encoding="utf-8") as log:
                        log.write(f"No doc_id mapped for index {idx} at {datetime.now()}\n")
                    continue

                if doc_id in doc_data:
                    try:
                        sim_score = similarities[idx]
                        if 0.4 < sim_score < 0.999:
                            processed_query_text = ast.literal_eval(doc_data[doc_id]) if doc_data[doc_id] else []
                            if not processed_query_text:
                                print(f"Warning: Empty processed_text for doc_id {doc_id}")
                                with open(log_file, "a", encoding="utf-8") as log:
                                    log.write(f"Empty processed_text for doc_id {doc_id} at {datetime.now()}\n")
                                continue
                            print(f"Selected document at doc_id {doc_id}, similarity: {sim_score}, text: {processed_query_text}")
                            with open(log_file, "a", encoding="utf-8") as log:
                                log.write(f"Selected document at doc_id {doc_id}, similarity: {sim_score}, text: {processed_query_text} at {datetime.now()}\n")
                            similar_queries.append(processed_query_text)
                            top_doc_words.extend(processed_query_text)
                            added += 1
                            if added >= max_add:
                                break
                    except (ValueError, SyntaxError) as e:
                        print(f"Error parsing processed_text at doc_id {doc_id}: {str(e)}")
                        with open(log_file, "a", encoding="utf-8") as log:
                            log.write(f"Error parsing processed_text at doc_id {doc_id}: {str(e)} at {datetime.now()}\n")
                        continue
                else:
                    print(f"Warning: doc_id {doc_id} not found in doc_data")

            if not similar_queries:
                print(f"No similar documents found for {dataset_name} with sufficient similarity")
                with open(log_file, "a", encoding="utf-8") as log:
                    log.write(f"No similar documents found for {dataset_name} with sufficient similarity at {datetime.now()}\n")
                return query, processed_query

            new_words = []
            seen = set(processed_query)
            for word in top_doc_words:
                if word not in seen and word not in new_words:
                    new_words.append(word)
                    if len(new_words) >= 4:
                        break 

            additional_words = ' '.join(new_words)
            print(f"Additional words from top 5 documents: {additional_words}")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"Additional words for {dataset_name}: {additional_words} at {datetime.now()}\n")

            if additional_words:
                enhanced_query = query + ' ' + additional_words
                print(f"🔍 Enhanced Query: {enhanced_query}")
                with open(log_file, "a", encoding="utf-8") as log:
                    log.write(f"Enhanced Query for {dataset_name}: {enhanced_query} at {datetime.now()}\n")
            else:
                enhanced_query = query
                print(f"No new words to add to query for {dataset_name}")
                with open(log_file, "a", encoding="utf-8") as log:
                    log.write(f"No new words to add to query for {dataset_name} at {datetime.now()}\n")

        except Exception as e:
            print(f"Error enhancing query for {dataset_name}: {str(e)}")
            with open(log_file, "a", encoding="utf-8") as log:
                log.write(f"Error enhancing query for {dataset_name}: {str(e)} at {datetime.now()}\n")
            if 'conn' in locals():
                conn.close()
            enhanced_query = query

        return enhanced_query, processed_query

    return query, processed_query

def process_csv(input_csv, output_csv, dataset_name, model):
    # Ensure logs directory exists
    os.makedirs("logs", exist_ok=True)
    
    # Read the input CSV
    try:
        df = pd.read_csv(input_csv)
        print(f"Loaded CSV file: {input_csv}")
    except Exception as e:
        print(f"Error reading CSV file {input_csv}: {str(e)}")
        return

    # Ensure the text column exists
    if 'text' not in df.columns:
        print("Error: 'text' column not found in CSV")
        return

    # Process each query with tqdm progress bar
    enhanced_queries = []
    processed_queries = []
    
    for query in tqdm(df['text'], desc="معالجة الاستعلامات", unit="استعلام"):
        enhanced_query, processed_query = enhance_query(query, dataset_name, model, enhance=True)
        enhanced_queries.append(enhanced_query)
        processed_queries.append(' '.join(processed_query))

    # Create a new DataFrame with the same columns
    result_df = df.copy()
    result_df['text'] = enhanced_queries
    result_df['processed_query'] = processed_queries

    # Save to output CSV
    try:
        result_df.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"Enhanced queries saved to {output_csv}")
    except Exception as e:
        print(f"Error saving CSV file {output_csv}: {str(e)}")

# Example usage
if __name__ == "__main__":
    from sentence_transformers import SentenceTransformer
    
    # Load the model
    model = SentenceTransformer('all-mpnet-base-v2')
    dataset_name = "antique"
    input_csv = r"data\antique\queries_antique.csv"
    output_csv = "enhanced_queries.csv"
    
    process_csv(input_csv, output_csv, dataset_name, model)

Loaded CSV file: data\antique\queries_antique.csv


معالجة الاستعلامات:   0%|          | 0/176 [00:00<?, ?استعلام/s]

Top similarities: [0.7374319  0.7374319  0.7328346  0.72237104 0.6528705 ]
Loaded doc_id_mapping with 403666 entries


معالجة الاستعلامات:   1%|          | 1/176 [00:04<11:49,  4.06s/استعلام]

Total documents in doc_data: 403666
Sample doc_ids: ['2020338_0', '2020338_1', '2020338_2', '2020338_3', '2874684_0']
Top indices: [288355 256467 208243 138791 281119]
Selected document at doc_id 3265991_12, similarity: 0.7374318838119507, text: ['concentration']
Selected document at doc_id 2036065_1, similarity: 0.7374318838119507, text: ['concentration']
Selected document at doc_id 1253368_5, similarity: 0.7328345775604248, text: ['concentrate', 'mind']
Selected document at doc_id 971642_3, similarity: 0.7223710417747498, text: ['concentrate']
Selected document at doc_id 4087614_7, similarity: 0.6528704762458801, text: ['try', 'yoga', 'concentration']
Additional words from top 5 documents: concentrate mind try yoga
🔍 Enhanced Query: how can we get concentration onsomething? concentrate mind try yoga
Top similarities: [0.37147337 0.3650272  0.36356586 0.35125536 0.35112196]
Loaded doc_id_mapping with 403666 entries


...

Total documents in doc_data: 403666
Sample doc_ids: ['2020338_0', '2020338_1', '2020338_2', '2020338_3', '2874684_0']
Top indices: [258555 149312 353393 357371 353363]
Selected document at doc_id 298001_7, similarity: 0.5947680473327637, text: ['taco', 'bell']
Selected document at doc_id 3164414_13, similarity: 0.5947679281234741, text: ['taco', 'bell']
Selected document at doc_id 4406669_41, similarity: 0.5777415037155151, text: ['taco']
Selected document at doc_id 9560_3, similarity: 0.5732377767562866, text: ['taco', 'bell', 'mcdonalds']
Selected document at doc_id 4406669_11, similarity: 0.5612599849700928, text: ['burritoes', 'tacos']
Additional words from top 5 documents: mcdonalds burritoes tacos
🔍 Enhanced Query: What do you order at Taco Bell? mcdonalds burritoes tacos
Top similarities: [0.59091467 0.5347402  0.52927303 0.52641034 0.5242784 ]
Loaded doc_id_mapping with 403666 entries


معالجة الاستعلامات:  99%|█████████▉| 175/176 [09:20<00:03,  3.18s/استعلام]

Total documents in doc_data: 403666
Sample doc_ids: ['2020338_0', '2020338_1', '2020338_2', '2020338_3', '2874684_0']
Top indices: [265804  93157  41937 368202 129576]
Selected document at doc_id 3108202_2, similarity: 0.5909146666526794, text: ['school', 'use']
Selected document at doc_id 4334121_1, similarity: 0.5347402095794678, text: ['real', 'world']
Selected document at doc_id 658351_9, similarity: 0.5292730331420898, text: ['school', 'systems']
Selected document at doc_id 1835434_13, similarity: 0.5264103412628174, text: ['seriousdid', 'school']
Selected document at doc_id 1582877_7, similarity: 0.5242784023284912, text: ['real', 'world', 'live']
Additional words from top 5 documents: systems seriousdid live
🔍 Enhanced Query: why do we go to school if in the real world we dont use most of it? systems seriousdid live
Top similarities: [0.49015477 0.47434306 0.47397465 0.46743023 0.4659585 ]
Loaded doc_id_mapping with 403666 entries


معالجة الاستعلامات: 100%|██████████| 176/176 [09:23<00:00,  3.20s/استعلام]

Total documents in doc_data: 403666
Sample doc_ids: ['2020338_0', '2020338_1', '2020338_2', '2020338_3', '2874684_0']
Top indices: [  5425 268205 264961 321528 255213]
Selected document at doc_id 4250745_1, similarity: 0.49015477299690247, text: ['christmas', 'become', 'cultural', 'holiday', 'associate', 'spend', 'time', 'family', 'christian', 'holiday', 'also', 'government', 'holiday', 'way', 'tradition', 'american', 'culture', 'spend', 'time', 'families', 'church', 'give', 'idea', 'christmas', 'encourage', 'close', 'christmas', 'sunday', 'church', 'leader', 'tradition', 'always', 'worship', 'sunday', 'always', 'worship', 'christmas', 'church', 'open', 'doesnt', 'make', 'better', 'anyone', 'else', 'see', 'christmas', 'holiday', 'church', 'rather', 'family', 'holiday', 'like', 'mother', 'day', 'something']
Selected document at doc_id 1122680_2, similarity: 0.47434306144714355, text: ['hear', 'lady', 'radio', 'day', 'say', 'love', 'christmas', 'love', 'tree', 'gift', 'season', 'santa', 